<a href="https://colab.research.google.com/github/debkrishna/CD-estimator-Chambers-Dunstan--Advanced-sample-survey-/blob/main/Updating_the_prior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import psycopg2
import numpy as np
import pandas as pd
from psycopg2.extras import DictCursor
import pymc3 as pm
from pymc3.distributions import Interpolated
import theano.tensor as tt
from scipy import stats
from model import create_submodel_1, create_submodel_2
from training import train_submodel_1,train_submodel_2,get_situation,get_outcome,initialise

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
def get_queryset(league = 'ICC T20'):
    """
    get queryset for model from database.
    match_id - id of the match
    inning - match inning
    team1_id - batting team 1 id
    team2_id - bowling team 2 id
    over - over of the ball
    ball - ball of the inning
    batsmen_id - id of batsmen
    non_striker_id - id of non striker batsmen
    bowler_id - id of the bowler
    is_wide - True/False
    is_no_ball - True/False
    total_runs - total run till now
    is_out - True/False
    :return:
    """
    connection = psycopg2.connect(
        database="cricket",
        user="postgres",
        password="ynKrco0QWa56iez8sKtG",
        host="trusted-prediction-research.c8qtxwbvasgw.ap-south-1.rds.amazonaws.com",
        port="5432"
    )
    cursor = connection.cursor(cursor_factory=DictCursor)
    cursor.execute(
        """
        select
            match_id, inning, css.team_id, opp_team_id, com.over,
            ball, batsmen_id, offstrike_batsmen_id non_striker_id,
            bowler_id, is_wide, is_no_ball, is_out, is_caugth,
            player_out_id, com.runs, total_runs
        from cds_commentary_commentary com
        inner join cds_scorecard_scorecard css on com.scorecard_id = css.id
        inner join cds_match_match cmm on css.match_id = cmm.id
        inner join cds_series_series c on cmm.series_id = c.id
        inner join cds_league_league cll on c.league_id = cll.id
        where cll.name = 'ICC T20'
        limit 1000
        """.format(league=league)
    )
    queryset = cursor.fetchall()
    connection.close()
    return queryset

In [ ]:
def load_data():
    '''
    Load data from queryset
    '''
    queryset = get_queryset()
    result = []
    for row in queryset:
        result.append(dict(row))
    return result

### All previous matches Data upto the current date.
#### But we should consider only recent matches to capture the updated likelihood. 

In [ ]:
def train_submodel_1_new(n_iter, target_accept, I, J, id1, id2, X, save_directory):
    '''
    Initialises and trains submodel 1.
    Creates save directory if it doesn't exist.
    '''
    wickets_sub_model = create_submodel_1(I, J, id1, id2, X)

    with wickets_sub_model:
        trace = pm.sample(n_iter, target_accept=target_accept, cores=1)

    cutpoints1 = np.mean(trace.get_values("cutpoints1", burn=n_iter // 2, combine=True), axis=0)
    mu1 = np.mean(trace.get_values("mu1", burn=n_iter // 2, combine=True), axis=0)
    mu2 = np.mean(trace.get_values("mu2", burn=n_iter // 2, combine=True), axis=0)
    delta11 = np.mean(trace.get_values("delta11", burn=n_iter // 2, combine=True), axis=0)
    delta12 = np.mean(trace.get_values("delta12", burn=n_iter // 2, combine=True), axis=0)
    delta13 = np.mean(trace.get_values("delta13", burn=n_iter // 2, combine=True), axis=0)
    delta1 = [delta11, delta12, delta13, delta11, delta12, delta13, delta11, delta12, delta13]

    cutpoints1_values = trace.get_values("cutpoints1", burn=n_iter // 2, combine=True)
    mu1_values = trace.get_values("mu1", burn=n_iter // 2, combine=True)
    mu2_values = trace.get_values("mu2", burn=n_iter // 2, combine=True)
    delta11_values = trace.get_values("delta11", burn=n_iter // 2, combine=True)
    delta12_values = trace.get_values("delta12", burn=n_iter // 2, combine=True)
    delta13_values = trace.get_values("delta13", burn=n_iter // 2, combine=True)
    delta1_values = [delta11_values, delta12_values, delta13_values, delta11_values, delta12_values, delta13_values, delta11_values, delta12_values, delta13_values]
    

    return delta1, mu1, mu2, cutpoints1,delta11_values, delta12_values, delta13_values,mu1_values, mu2_values, cutpoints1_values


In [ ]:
target_accept = 0.95  # doesn't need to be changed
n_iter = 500
save_directory = "save"
deliveries_data = load_data()

batsmen, bowlers, batsman_index, bowler_index, X, id1, id2, noballs_and_wides = initialise(deliveries_data[:954])

A = train_submodel_1_new(n_iter, target_accept, len(batsmen), len(bowlers), id1, id2, X,
                 save_directory)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [cutpoints1, mu2, mu1, tau2, tau1, delta13, delta12, delta11]
100%|██████████| 1000/1000 [00:50<00:00, 19.82it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


In [ ]:
Hist_data = initialise(deliveries_data[:954])
part_data = deliveries_data[953:]
new_match_data_ini  = initialise(part_data)

In [ ]:
batsman_present = []
for i in range(len(new_match_data_ini[0])):
  if (new_match_data_ini[0][i] in Hist_data[0]) == True:
    batsman_present.append(new_match_data_ini[0][i])

In [ ]:
bowler_present = []
for i in range(len(new_match_data_ini[1])):
  if (new_match_data_ini[1][i] in Hist_data[1]) == True:
    bowler_present.append(new_match_data_ini[1][i])

In [ ]:
p_dt = pd.DataFrame(part_data)
n_m_d = p_dt[(p_dt.batsmen_id.isin(batsman_present)) & (p_dt.bowler_id.isin(bowler_present))]

In [ ]:
n_m_d_l = list(n_m_d.to_dict(orient='records'))
new_match_data  = initialise(n_m_d_l)

In [ ]:
def transformed_index(Hist_data,new_match_data):
    trans_batsman_index = {}
    for i in new_match_data[0]:
        batsman_index = Hist_data[2]
        trans_batsman_index[i] = batsman_index.get(i)
    trans_bowler_index = {}
    for i in new_match_data[1]:
        bowler_index = Hist_data[3]
        trans_bowler_index[i] = bowler_index.get(i)
    for i in range(len(new_match_data[5])):
        for j in range(len((new_match_data[5])[i])):
            new_value = (new_match_data[5])[i][j]
            k = list(new_match_data[2].keys())[list(new_match_data[2].values()).index(new_value)]
            (new_match_data[5])[i][j] = trans_batsman_index[k]  
    for i in range(len(new_match_data[6])):
        for j in range(len((new_match_data[6])[i])):
            new_value = (new_match_data[6])[i][j]
            k = list(new_match_data[3].keys())[list(new_match_data[3].values()).index(new_value)]
            (new_match_data[6])[i][j] = trans_bowler_index[k]
            # id1 and id2 of new_match_data will be transformed
    return(trans_batsman_index,trans_bowler_index,new_match_data)

In [ ]:

def from_posterior(param, samples):
    smin, smax = np.min(samples), np.max(samples)
    width = smax - smin
    x = np.linspace(smin, smax, 100)
    y = stats.gaussian_kde(samples)(x)#we can use gridsearch for best fitting kernal

    x = np.concatenate([[x[0] - 3 * width], x, [x[-1] + 3 * width]])
    y = np.concatenate([[0], y, [0]])
    return Interpolated(param, x, y)

In [ ]:
trans_batsmen = list(transformed_index(Hist_data,new_match_data)[0].values())
trans_bowlers = list(transformed_index(Hist_data,new_match_data)[1].values())


In [ ]:
def updated_submodel_1(trans_batsmen,trans_bowlers, id1, id2, X):
    '''
    Submodel 1: models the fall of wickets.
    '''
    X1 = [[1 if [l][i] == 1 else 2 for i in range(len(X[l]))] for l in range(9)]
    X1 = np.asarray([np.array(X1[i]) for i in range(9)], dtype=object)
    new_model = pm.Model()

    with new_model:
        delta11 = (from_posterior("delta11",A[4] ))
        delta12 = (from_posterior("delta12", A[5]))
        delta13 = (from_posterior("delta13", A[6]))
        delta1 = [delta11, delta12, delta13, delta11, delta12, delta13, delta11, delta12, delta13]
        mu1 = ([from_posterior("mu1"+ str(i),A[7][:,i]) for i in trans_batsmen])
        mu2 = ([from_posterior("mu2"+ str(i),A[8][:,i]) for i in trans_bowlers])
        mu1 = tt.as_tensor_variable(mu1)
        mu2 = tt.as_tensor_variable(mu2)
        eta1 = [pm.Deterministic("eta1_" + str(i), delta1[i] + mu1[id1[i]] - mu2[id2[i]]) for i in range(9)]

        cutpoints1 = from_posterior("cutpoints1",A[9].reshape(-1))

        X1_ = [pm.OrderedLogistic("X1_" + str(i), cutpoints=cutpoints1, eta=eta1[i], observed=X1[i] - 1) for i in
               range(9)]
        step = pm.Metropolis()
        updated_trace = pm.sample(step = step)
        mu1_est_values = [updated_trace["mu1"+str(i)] for i in trans_batsmen]
        mu2_est_values = [updated_trace["mu2"+str(i)] for i in trans_bowlers]
        mu1_est = np.array(mu1_est_values).mean(axis = 1)
        mu2_est = np.array(mu2_est_values).mean(axis = 1)
        delta11_est = updated_trace["delta11"].mean()
        delta12_est = updated_trace["delta12"].mean()
        delta13_est = updated_trace["delta13"].mean()
        cutpoints1_est= updated_trace["cutpoints1"].mean()
        delta1_est = [delta11_est, delta12_est, delta13_est, delta11_est, delta12_est, delta13_est, delta11_est, delta12_est, delta13_est]
    return delta1_est, mu1_est, mu2_est, cutpoints1_est


In [ ]:
X = new_match_data[4]
S = updated_submodel_1(trans_batsmen,trans_bowlers, new_match_data[5], new_match_data[6],X)


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [cutpoints1]
>Metropolis: [mu213]
>Metropolis: [mu123]
>Metropolis: [mu141]
>Metropolis: [delta13]
>Metropolis: [delta12]
>Metropolis: [delta11]
100%|██████████| 1000/1000 [00:02<00:00, 403.21it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


In [ ]:
def train_submodel_2_new(n_iter, target_accept, I, J, id1, id2, X, save_directory):
    '''
    Initialises and trains submodel 2.
    Creates save directory if it doesn't exist.
    '''
    wickets_sub_model = create_submodel_2(I, J, id1, id2, X)

    with wickets_sub_model:
        trace = pm.sample(n_iter, target_accept=target_accept, cores=1)

    cutpoints2 = np.mean(trace.get_values("cutpoints2", burn=n_iter // 2, combine=True), axis=0)
    mu3 = np.mean(trace.get_values("mu3", burn=n_iter // 2, combine=True), axis=0)
    mu4 = np.mean(trace.get_values("mu4", burn=n_iter // 2, combine=True), axis=0)
    delta21 = np.mean(trace.get_values("delta21", burn=n_iter // 2, combine=True), axis=0)
    delta22 = np.mean(trace.get_values("delta22", burn=n_iter // 2, combine=True), axis=0)
    delta2 = np.greater_equal([i for i in range(9)], 3) * delta21 + np.greater_equal([i for i in range(9)], 6) * delta22
    cutpoints2_values = trace.get_values("cutpoints2", burn=n_iter // 2, combine=True)
    mu3_values = trace.get_values("mu3", burn=n_iter // 2, combine=True)
    mu4_values = trace.get_values("mu4", burn=n_iter // 2, combine=True)
    delta21_values = trace.get_values("delta21", burn=n_iter // 2, combine=True)
    delta22_values = trace.get_values("delta22", burn=n_iter // 2, combine=True)

    return delta2, mu3, mu4, cutpoints2,delta21_values, delta22_values,mu3_values, mu4_values, cutpoints2_values


In [ ]:
target_accept = 0.95  # doesn't need to be changed
n_iter = 500
save_directory = "save"
deliveries_data = load_data()

batsmen, bowlers, batsman_index, bowler_index, X, id1, id2, noballs_and_wides = initialise(deliveries_data)

B = train_submodel_2_new(n_iter, target_accept, len(batsmen), len(bowlers), id1, id2, X,
                 save_directory)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [cutpoints2, mu4, mu3, tau2, tau1, delta22, delta21]
100%|██████████| 1000/1000 [00:36<00:00, 27.27it/s]
The number of effective samples is smaller than 25% for some parameters.


In [ ]:
def myfilter(X, A):
    '''
    Helper function to filter parameters in submodel 2
    '''
    ret = []
    for l in range(9):
        tmp = []
        for i in range(len(X[l])):
            if X[l][i] != 1:
                tmp.append(A[l][i])
        ret.append(tmp)
    ret = np.asarray([np.array(ret[i]) for i in range(9)], dtype=object)
    return ret

In [ ]:
def updated_submodel_2(trans_batsmen,trans_bowlers, id1, id2, X):
    '''
    Submodel 2: models runs scored.
    '''
    X1 = [[1 if X[l][i] == 1 else 2 for i in range(len(X[l]))] for l in range(9)]
    X1 = np.asarray([np.array(X1[i]) for i in range(9)], dtype=object)
    id3 = myfilter(X, id1)
    id4 = myfilter(X, id2)
    X2 = myfilter(X, X) - 1
    l = [i for i in range(9)]
    new_model = pm.Model()
    with new_model:
        delta21 = (from_posterior("delta21",B[4] ))
        delta22 = (from_posterior("delta22", B[5]))
        delta2 = pm.math.ge(l, 3) * delta21 + pm.math.ge(l, 6) * delta22  # offset for different situations
        mu3 = ([from_posterior("mu3"+ str(i),B[6][:,i]) for i in trans_batsmen])
        mu4 = ([from_posterior("mu4"+ str(i),B[7][:,i]) for i in trans_bowlers])
        mu3 = tt.as_tensor_variable(mu3)
        mu4 = tt.as_tensor_variable(mu4)
        eta2 = [pm.Deterministic("eta2_" + str(i), delta2[i] + mu3[id3[i]] - mu4[id4[i]]) for i in range(9)]
        
        cutpoints2 = [[from_posterior("cutpoints2"+str(j)+str(i),B[8][:,j,i]) for i in range(5)] for j in range(9)]

        X2_ = [pm.OrderedLogistic("X2_" + str(i), cutpoints=cutpoints2[i], eta=eta2[i], observed=X2[i] - 1) for i in
               range(9)]
        step = pm.Metropolis()
        updated_trace = pm.sample(step = step)
        mu3_est_values = [updated_trace["mu3"+str(i)] for i in trans_batsmen]
        mu4_est_values = [updated_trace["mu4"+str(i)] for i in trans_bowlers]
        mu3_est = np.array(mu3_est_values).mean(axis = 1)
        mu4_est = np.array(mu4_est_values).mean(axis = 1)
        delta21_est_values = updated_trace["delta21"]
        delta22_est_values = updated_trace["delta22"]
        delta21_est = updated_trace["delta21"].mean()
        delta22_est = updated_trace["delta22"].mean()
        cutpoints2_est_values = [[updated_trace["cutpoints2"+str(j)+str(i)] for i in range(5)] for j in range(9)]
        cutpoints2_est = np.array(cutpoints2_est_values).mean(axis = 2)
        delta2_est = np.greater_equal([i for i in range(9)], 3) * delta21_est + np.greater_equal([i for i in range(9)], 6) * delta22_est
    return delta2_est, mu3_est, mu4_est,cutpoints2_est,delta21_est_values,delta22_est_values,mu3_est_values,mu4_est_values,cutpoints2_est_values


In [ ]:
X = new_match_data[4]
R = updated_submodel_2(trans_batsmen,trans_bowlers, new_match_data[5], new_match_data[6],X)

Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [cutpoints284]
>Metropolis: [cutpoints283]
>Metropolis: [cutpoints282]
>Metropolis: [cutpoints281]
>Metropolis: [cutpoints280]
>Metropolis: [cutpoints274]
>Metropolis: [cutpoints273]
>Metropolis: [cutpoints272]
>Metropolis: [cutpoints271]
>Metropolis: [cutpoints270]
>Metropolis: [cutpoints264]
>Metropolis: [cutpoints263]
>Metropolis: [cutpoints262]
>Metropolis: [cutpoints261]
>Metropolis: [cutpoints260]
>Metropolis: [cutpoints254]
>Metropolis: [cutpoints253]
>Metropolis: [cutpoints252]
>Metropolis: [cutpoints251]
>Metropolis: [cutpoints250]
>Metropolis: [cutpoints244]
>Metropolis: [cutpoints243]
>Metropolis: [cutpoints242]
>Metropolis: [cutpoints241]
>Metropolis: [cutpoints240]
>Metropolis: [cutpoints234]
>Metropolis: [cutpoints233]
>Metropolis: [cutpoints232]
>Metropolis: [cutpoints231]
>Metropolis: [cutpoints230]
>Metropolis: [cutpoints224]
>Metropolis: [cutpoints223]
>Metropolis: [cutpoints222]
>Metropolis: [cutpoints